# MiniProject02
 This is our implementation for COMP 551 miniproject 02

In [158]:
import numpy as np
import matplotlib.pyplot as plt
import math 
import pandas as pd
import json
import time
import os 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer 
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re

## Build feature vector for instance


In [159]:
from sklearn.model_selection import train_test_split
# directory for training data
dir = 'C:/Users/Hassan/Documents/Semester 6/COMP 551/Projects/proj2_materials/train/'
text_neg = [] # list of all negative reviews in training set
text_pos = [] # list of all positive reviews in training set


# load the training data
for filename in os.listdir(dir + '/neg'):
        with open(dir + '/neg/' +  filename, 'r',encoding="utf8") as f:
            text_neg.append(f.read().split())
            
for filename in os.listdir(dir + '/pos'):
        with open(dir + '/pos/' +  filename, 'r',encoding="utf8") as f:
            text_pos.append(f.read().split())

# directory for testing data
dir_test  = 'C:/Users/Hassan/Documents/Semester 6/COMP 551/Projects/proj2_materials/test/'
text_test = []
# load testing data         
for i in range(len(os.listdir(dir_test))):
    with open('{}{}.txt'.format(dir_test, i), 'r',encoding="utf8") as f:
            text_test.append(f.read().split()) 
            
            
            
y_pos = [1] * len(text_pos)
y_neg = [0] * len(text_neg)

X_train = text_neg + text_pos

In [161]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# not used
def preprocess_text(text):
    text = re.sub("<*>", ' ', text)
    text = text.lower().replace('\n', ' ').split()
    for i in range(len(text)):
        text[i] = lemmatizer.lemmatize(text[i])
    return text

# for i in range(len(X_train)):
#   X_train[i] = preprocess_text(X_train[i])
    



y_neg = y_neg + y_pos 
y= y_neg

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, train_size=0.8, test_size=0.2)

In [162]:


# some text processing of data
new_x = list()
for i in range(len(X_train)):
    new_x.append(" ".join(X_train[i]))
    
new_x_valid = list ()
for i in range(len(X_valid)):
    new_x_valid.append(" ".join(X_valid[i]))
    
new_x_test = list()
for i in range(len(text_test)):
    new_x_test.append(" ".join(text_test[i]))

## Create Pipeline, train model, and pass in validation set

In [163]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
import csv


from functools import reduce
import itertools
import operator


X_train = new_x
X_test = new_x_test
X_valid = new_x_valid

# prediction pipeline
pclf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf',  LinearSVC()),
])


# train model
pclf.fit(X_train, y_train) 

# predict training
prediction_train = pclf.predict(X_train)

# predict validation
prediction_val = pclf.predict(X_valid)

print(classification_report(y_valid, prediction_val))


             precision    recall  f1-score   support

          0       0.90      0.88      0.89      2488
          1       0.89      0.90      0.89      2512

avg / total       0.89      0.89      0.89      5000



In [164]:
print(classification_report(y_train, prediction_train))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99     10012
          1       0.99      0.99      0.99      9988

avg / total       0.99      0.99      0.99     20000



In [168]:
X_train = new_x
X_test = new_x_test
X_valid = new_x_valid

# prediction pipeline
pclf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf',  DecisionTreeClassifier()),
])


# train model
pclf.fit(X_train, y_train) 

# predict training
prediction_train = pclf.predict(X_train)

# predict validation
prediction_val = pclf.predict(X_valid)

print(classification_report(y_valid, prediction_val))


             precision    recall  f1-score   support

          0       0.90      0.88      0.89      2488
          1       0.89      0.90      0.89      2512

avg / total       0.89      0.89      0.89      5000



In [ ]:
X_train = new_x
X_test = new_x_test
X_valid = new_x_valid

# prediction pipeline
pclf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf',  LogisticRegression()),
])


# train model
pclf.fit(X_train, y_train) 

# predict training
prediction_train = pclf.predict(X_train)

# predict validation
prediction_val = pclf.predict(X_valid)

print(classification_report(y_valid, prediction_val))


## Predictions for testing set

In [165]:
import pandas as pd 
prediction_test = pclf.predict(X_test) 
data_to_submit = pd.DataFrame({'Id': [i for i in range(len(prediction_test))], 'Category': prediction_test })
data_to_submit.to_csv("C:/Users/Hassan/Documents/Semester 6/COMP 551/Projects/proj2_materials/file.csv", index = False)


In [166]:
prediction_test

array([0, 0, 0, ..., 1, 1, 0])

## Cross Validation
In this section, we wish to add the k-fold cross validation to our pipeline

In [167]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as randint

params = {"vect__ngram_range": [(1,1),(1,2),(2,2)],
          "tfidf__use_idf": [True, False]}

seed = 551 # Very important for repeatibility in experiments!

random_search = RandomizedSearchCV(pclf, param_distributions = params, cv=2, verbose = 10, random_state = seed, n_iter = 6)
random_search.fit(X_train, y_train)


Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=True ...................
[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, score=0.886, total=   5.3s
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=True ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s remaining:    0.0s


[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, score=0.8822, total=   6.7s
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=True ...................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.2s remaining:    0.0s


[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=True, score=0.8971, total=  20.9s
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=True ...................


...

[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=False, score=0.8833, total=  19.7s
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False ..................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.9min remaining:    0.0s


[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=False, score=0.8796, total=  19.5s
[CV] vect__ngram_range=(2, 2), tfidf__use_idf=False ..................
[CV]  vect__ngram_range=(2, 2), tfidf__use_idf=False, score=0.8616, total=  14.4s
[CV] vect__ngram_range=(2, 2), tfidf__use_idf=False ..................
[CV]  vect__ngram_range=(2, 2), tfidf__use_idf=False, score=0.8546, total=  13.5s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  4.0min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
          fit_params=None, iid=True, n_iter=6, n_jobs=1,
          param_distributions={'vect__ngram_range': [(1, 1), (1, 2), (2, 2)], 'tfidf__use_idf': [True, False]},
          pre_dispatch='2*n_jobs', random_state=551, refit=True,
          return_train_score='warn', scoring=None, verbose=10)